In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import find_peaks, savgol_filter
import re
import csv
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
import itertools
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from pathlib import Path
from scipy.io import loadmat 
from pathlib import Path

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

# Leer los datos de todos los archivos .mat en todas las subcarpetas de 0km_0dBm
def read_all_csv_data(main_folder_path):
    data = {}

    base_path = os.path.join(main_folder_path, "3x32Gbaud_16QAM", "270km_9dBm")

    if not (os.path.exists(base_path) and os.path.isdir(base_path)):
        print(f"La ruta base '{base_path}' no existe o no es válida.")
        return data

    # Recorrer todas las subcarpetas dentro de 0km_0dBm
    for subfolder in os.listdir(base_path):
        subfolder_path = os.path.join(base_path, subfolder)

        if os.path.isdir(subfolder_path):
            key_path = f"3x32Gbaud_16QAM > 270km_9dBm > {subfolder}"
            data[key_path] = []

            # Procesar archivos .mat en cada subcarpeta
            for filename in os.listdir(subfolder_path):
                if filename.endswith(".mat"):
                    file_path = os.path.join(subfolder_path, filename)
                    mat_data = loadmat(file_path)

                    if "rconst" in mat_data:
                        df = mat_data["rconst"][0]
                        df = pd.DataFrame({"I": df.real, "Q": df.imag})
                        data[key_path].append((filename, df))
                    else:
                        print(f"Advertencia: 'rconst' no encontrado en el archivo {filename} en {subfolder_path}")
    return data

# Encontrar picos en un segmento
def find_peak_in_segment(counts, bin_edges, start, end):
    start_idx = np.searchsorted(bin_edges, start, side='left')
    end_idx = np.searchsorted(bin_edges, end, side='left')

    if start_idx >= end_idx or start_idx >= len(counts):
        return None  # Segmento vacío o fuera de límites

    peak_idx = start_idx + np.argmax(counts[start_idx:end_idx])
    return peak_idx

# Encontrar valles en un segmento
def find_valley_in_segment(counts, bin_edges, start, end):
    start_idx = np.searchsorted(bin_edges, start, side='right')
    end_idx = np.searchsorted(bin_edges, end, side='left')

    if start_idx >= end_idx or start_idx >= len(counts):
        return None  # Segmento vacío o fuera de límites

    valley_idx = start_idx + np.argmin(counts[start_idx:end_idx])
    return valley_idx



# Ruta principal y ejecución
main_folder_path = "/Users/USUARIO/Documents/ProyectoJames"
data = read_all_csv_data(main_folder_path)


    


In [3]:
# Graficar histogramas para todos los archivos en todas las subcarpetas
def plot_histograms_for_all(data, var_name):
    results_all = {}  # Inicializar siempre el diccionario de resultados
    for key_path, files in data.items():
        print(f"Procesando carpeta: {key_path}")

        for item in files:
            if len(item) != 2:
                print(f"Error: se esperaba una tupla con 2 elementos, pero se encontró: {item}")
                continue

            filename, df = item

            counts, bin_edges = np.histogram(df[var_name], bins=100)

            peaks = [
                find_peak_in_segment(counts, bin_edges, -4, -2),
                find_peak_in_segment(counts, bin_edges, -2, 0),
                find_peak_in_segment(counts, bin_edges, 0, 2),
                find_peak_in_segment(counts, bin_edges, 2, 4)
            ]

            valleys = [
                find_valley_in_segment(counts, bin_edges, -3, -1),
                find_valley_in_segment(counts, bin_edges, -1, 1),
                find_valley_in_segment(counts, bin_edges, 1, 3)
            ]

            peaks = [peak for peak in peaks if peak is not None]
            valleys = [valley for valley in valleys if valley is not None]

            peaks_bin_edges = bin_edges[peaks]
            valleys_bin_edges = bin_edges[valleys]
            ensanchamiento = bin_edges[-1] - bin_edges[0]

            # Guardar los resultados
            results_all[filename] = {
                'peaks': peaks_bin_edges,
                'valleys': valleys_bin_edges,
                'ensanchamiento': ensanchamiento
            }


            #plt.figure(figsize=(10, 6))
            #plt.hist(df[var_name], bins=100, alpha=0.7, color='blue', edgecolor='black')
            #plt.plot(bin_edges[peaks], counts[peaks], 'ro', label='Picos')
            #plt.plot(bin_edges[valleys], counts[valleys], 'go', label='Valles')
            #plt.title(f'Histograma de {var_name} - {filename}')
            #plt.xlabel(f'Valor de {var_name}')
            #plt.ylabel('Frecuencia')
            #plt.legend()
            #plt.grid(True)
            #plt.show()
    return results_all

In [4]:
#print(data.keys())

In [12]:
#len(data.keys())

17

In [6]:
#data['3x32Gbaud_16QAM > 270km_9dBm > 31p5GHz']

In [7]:
#len(data['3x32Gbaud_16QAM > 270km_9dBm > 31p5GHz'])

64

## Datos con el Spacing

In [8]:
def analyze_data(data):
    pattern = r'(\d+(\.\d+)?)GHz'
    pattern_osnr = r'(\d+(\.\d+)?)(?=dB(?!m))'
    
    matriz_caract = np.empty((0, 18))  # Inicializa la matriz vacía
    count = 0  # Contador para limitar a 3 histogramas

    for subfolder1, archivos in data.items():  # Iterar sobre cada carpeta de nivel 1
        print(f'Se está leyendo la carpeta de nivel 1: {subfolder1}')
        
        if 'dBm' in subfolder1:  # Verificar que la carpeta tenga 'dBm'
            print(f'Procesando carpeta: {subfolder1}')
            
            # Almacenar los resultados de los histogramas solo una vez para la subcarpeta
            resultsI = plot_histograms_for_all({subfolder1: archivos}, var_name="I")
            resultsQ = plot_histograms_for_all({subfolder1: archivos}, var_name="Q")
            
            # Iterar sobre los archivos de la subcarpeta
            for (filename, df), (resultI, resultQ) in zip(archivos, zip(resultsI.items(), resultsQ.items())):
                print(f'Archivo encontrado: {filename}')

                match = re.search(pattern, subfolder1.replace('p','.'))  # Extraer GHz desde el nombre de la carpeta
                spacing = float(match.group(1)) if match else None

                match = re.search(pattern_osnr, resultI[0].replace('p','.'))
                OSNR = float(match.group(1)) if match else None

                I = resultI[1]
                Q = resultQ[1]

                # Concatenar los resultados de picos, valles y ensanchamiento
                vector = np.hstack((
                    I["peaks"], Q["peaks"],
                    I["valleys"], Q["valleys"],
                    I["ensanchamiento"],
                    Q["ensanchamiento"],
                    [spacing, OSNR]
                ))

                # Agregar los resultados a la matriz
                matriz_caract = np.vstack((matriz_caract, vector))
                count += 1  # Incrementa el contador

    return matriz_caract


In [9]:
matriz_caract = analyze_data(data)

Se está leyendo la carpeta de nivel 1: 3x32Gbaud_16QAM > 270km_9dBm > 29GHz
Procesando carpeta: 3x32Gbaud_16QAM > 270km_9dBm > 29GHz
Procesando carpeta: 3x32Gbaud_16QAM > 270km_9dBm > 29GHz
Procesando carpeta: 3x32Gbaud_16QAM > 270km_9dBm > 29GHz
Archivo encontrado: Song1_X_30p02dB_29GHz_270km_9dBm.mat
Archivo encontrado: Song1_Y_31p16dB_29GHz_270km_9dBm.mat
Archivo encontrado: Song2_X_24p03dB_29GHz_270km_9dBm.mat
Archivo encontrado: Song3_X_30p02dB_29GHz_270km_9dBm.mat
Se está leyendo la carpeta de nivel 1: 3x32Gbaud_16QAM > 270km_9dBm > 29p5GHz
Procesando carpeta: 3x32Gbaud_16QAM > 270km_9dBm > 29p5GHz
Procesando carpeta: 3x32Gbaud_16QAM > 270km_9dBm > 29p5GHz
Procesando carpeta: 3x32Gbaud_16QAM > 270km_9dBm > 29p5GHz
Archivo encontrado: Song1_X_26p06dB_29p5GHz_270km_9dBm.mat
Archivo encontrado: Song2_X_26p06dB_29p5GHz_270km_9dBm.mat
Archivo encontrado: Song2_X_30p11dB_29p5GHz_270km_9dBm.mat
Archivo encontrado: Song3_X_26p06dB_29p5GHz_270km_9dBm.mat
Archivo encontrado: Song3_X_29p1dB

In [10]:
matriz_caract.shape

(1096, 18)

In [11]:
# Convierte primero a DataFrame
df = pd.DataFrame(matriz_caract)


# Luego guarda
df.to_csv('matriz_caract.csv', index=False, header=False)